## Description
GMM is a very popular youtube series. They score and rank food very often and wanted to develop an script that extracts their scoring data from youtube easier than other more manual approaches

#### Youtube Transcript API
We first use the youtube transcript api to extract the video where GMM rates all the Tillamook icre creams

In [1]:

#--LIBRARIES--#
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

#--EXTRACT DATA FROM YOUTUBE--#
dicts = YouTubeTranscriptApi.get_transcript('cDKX3ClgbcI')


#### Filtering intro and outro
The rating of the videos occur in the middle of the episode, so I filtered out all the discussion occurring in the intro and outro.

In [2]:
#--FILTER THE INTRODUCTION OF THE VIDEO--#
filtered = [i for i in dicts if i['start'] >= 167.416 and i['start'] <950]

#### Filtering slang or exclamations
Because we are working with the raw conversation, then I have filtered out all the comments like Yeah, Oh, Nice, Oh okay etc. This helped cleaned the data

In [3]:
#--FILTER SLANG FROM THE TRANSCRIPT--#
filter_words = ['- Yeah.', '- Oh.', '- Nice.', '- Oh, okay.', '- Okay.', 'Eh!']

filtered2 = [i for i in filtered if i['text'] not in filter_words]


#### Extracting the names of the ice creams
Stevie, the exec producer of the show, is the one that mentions the ice creams to be reviewed. Then after a few back and forths, Rhett and Link provide a score out of 100. 

In [6]:
#--EXTRACT WHEN STEVIE TALKS-#
#This is usually when stevie mentions which ice_cream they are trying
stevie = [i for i in filtered2 if i['text'].startswith('- [Stevie]')]

stevie_df = pd.DataFrame(stevie, columns=['text', 'start', 'duration'])

#Extracting all of the automated [Stevie] script starts
stevie_df['text'] = stevie_df['text'].apply(lambda x: x.split('- [Stevie]')[-1].strip())

#Attempting to only leave the names of 90% of the ice creams
filter_strings = ['You tell me', 'Are you', 'It kind of', '84', "I'm sorry", 'Okay,', 'sAnd that', 'And that', 'We']

stevie_filtered = stevie_df[~stevie_df['text'].str.startswith(tuple(filter_strings))]

stevie_filtered['text'] = stevie_filtered['text'].str.rstrip('.')

/var/folders/1g/035gcyqx5rl66w96lpg8_54c5w0cnx/T/ipykernel_22709/2535776630.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stevie_filtered['text'] = stevie_filtered['text'].str.rstrip('.')


#### Finding the scores for each of those ice creams
